In [15]:
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm
import pickle
import os


## MIMIC - III, IV

In [2]:
III_DATA_PATH = '../train/mimiciii/'
III_lab = pd.read_csv(III_DATA_PATH + 'LABEVENTS.csv')
III_prescrip = pd.read_csv(III_DATA_PATH + 'PRESCRIPTIONS.csv')
III_input_cv = pd.read_csv(III_DATA_PATH + 'INPUTEVENTS_CV.csv')
III_input_mv = pd.read_csv(III_DATA_PATH + 'INPUTEVENTS_MV.csv')
III_icustay = pd.read_csv(III_DATA_PATH + 'ICUSTAYS.csv')
III_label = pd.read_csv('../train/labels/mimiciii_labels.csv')

In [3]:
IV_DATA_PATH = '../train/mimiciv/'
IV_lab = pd.read_csv(IV_DATA_PATH + 'labevents.csv')
IV_prescrip = pd.read_csv(IV_DATA_PATH + 'prescriptions.csv')
IV_input = pd.read_csv(IV_DATA_PATH + 'inputevents.csv')
IV_icustay = pd.read_csv(IV_DATA_PATH + 'icustays.csv')
IV_label = pd.read_csv('../train/labels/mimiciv_labels.csv')

/tmp/ipykernel_30132/333375981.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  IV_prescrip = pd.read_csv(IV_DATA_PATH + 'prescriptions.csv')


### ICUSTAY_ID 칼럼 추가

In [4]:
III_icustay.ICUSTAY_ID.value_counts().nlargest()

3027232    1
3001991    1
3002918    1
3010318    1
3013244    1
Name: ICUSTAY_ID, dtype: int64

In [5]:
IV_icustay.stay_id.value_counts().nlargest()

3034824    1
3038223    1
3049442    1
3042925    1
3021295    1
Name: stay_id, dtype: int64

In [6]:
III_lab['ICUSTAY_ID'] = int(0)
IV_lab['stay_id'] = int(0)

In [7]:
# III - lab
for i in tqdm(range(len(III_lab))): # i: lab index
    hadm_id = III_lab.HADM_ID.iloc[i]
    charttime = III_lab.CHARTTIME.iloc[i]
    idx = III_icustay[III_icustay.HADM_ID == hadm_id].index[0] # idx: icustay index
    intime = III_icustay.INTIME.iloc[idx]
    outtime = III_icustay.OUTTIME.iloc[idx]
    
    if datetime.strptime(intime,'%Y-%m-%d %H:%M:%S') <= datetime.strptime(charttime,'%Y-%m-%d %H:%M:%S') <= datetime.strptime(outtime,'%Y-%m-%d %H:%M:%S'):
        III_lab.ICUSTAY_ID.iloc[i] = III_icustay.ICUSTAY_ID.iloc[idx]

  0%|          | 0/1958619 [00:00<?, ?it/s]/tmp/ipykernel_30132/1142325161.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  III_lab.ICUSTAY_ID.iloc[i] = III_icustay.ICUSTAY_ID.iloc[idx]
100%|██████████| 1958619/1958619 [17:58<00:00, 1816.77it/s]


In [8]:
# IV - lab
for i in tqdm(range(len(IV_lab))): # i: lab index
    hadm_id = IV_lab.hadm_id.iloc[i]
    charttime = IV_lab.charttime.iloc[i]
    idx = IV_icustay[IV_icustay.hadm_id == hadm_id].index[0] # idx: icustay index
    intime = IV_icustay.intime.iloc[idx]
    outtime = IV_icustay.outtime.iloc[idx]
    
    if datetime.strptime(intime,'%Y-%m-%d %H:%M:%S') <= datetime.strptime(charttime,'%Y-%m-%d %H:%M:%S') <= datetime.strptime(outtime,'%Y-%m-%d %H:%M:%S'):
        IV_lab.stay_id.iloc[i] = IV_icustay.stay_id.iloc[idx]

  0%|          | 0/2996387 [00:00<?, ?it/s]/tmp/ipykernel_30132/2876006535.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  IV_lab.stay_id.iloc[i] = IV_icustay.stay_id.iloc[idx]
100%|██████████| 2996387/2996387 [28:18<00:00, 1764.42it/s]


In [9]:
IV_prescrip['stay_id'] = int(0)

In [10]:
# IV - Prescrip
for i in tqdm(range(len(IV_prescrip))): # i: lab index
    hadm_id = IV_prescrip.hadm_id.iloc[i]
    starttime = IV_prescrip.starttime.iloc[i]
    idx = IV_icustay[IV_icustay.hadm_id == hadm_id].index[0] # idx: icustay index
    intime = IV_icustay.intime.iloc[idx]
    outtime = IV_icustay.outtime.iloc[idx]
    
    if datetime.strptime(intime,'%Y-%m-%d %H:%M:%S') <= datetime.strptime(starttime,'%Y-%m-%d %H:%M:%S') <= datetime.strptime(outtime,'%Y-%m-%d %H:%M:%S'):
        IV_prescrip.stay_id.iloc[i] = IV_icustay.stay_id.iloc[idx]

  0%|          | 0/1298862 [00:00<?, ?it/s]/tmp/ipykernel_30132/2302662706.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  IV_prescrip.stay_id.iloc[i] = IV_icustay.stay_id.iloc[idx]
100%|██████████| 1298862/1298862 [12:22<00:00, 1748.73it/s]


### Concat the inputCV+inputMV for MIMIC-III

In [12]:
col = ['ICUSTAY_ID', 'ITEMID', 'AMOUNT', 'AMOUNTUOM','RATE', 'RATEUOM', 'ORIGINALAMOUNT', 'ORIGINALRATE', 'CHARTTIME']
III_input_mv.rename(columns = {'STARTTIME' : 'CHARTTIME'}, inplace = True)

cv = III_input_cv[col]
mv = III_input_mv[col]



III_input = pd.concat([cv, mv], axis=0).sort_values('ICUSTAY_ID')
III_input = III_input.reset_index(drop=True)
III_input

,ICUSTAY_ID,ITEMID,AMOUNT,AMOUNTUOM,RATE,RATEUOM,ORIGINALAMOUNT,ORIGINALRATE,CHARTTIME
0,3000000,30066,22.000000,ml,NaN,NaN,NaN,22.000000,2166-11-08 16:00:00
1,3000000,30131,NaN,NaN,60.000000,mcgkgmin,NaN,NaN,2166-11-08 17:00:00
2,3000000,30131,NaN,NaN,60.000000,mcgkgmin,NaN,NaN,2166-11-08 16:00:00
3,3000000,30131,NaN,NaN,60.000000,mcgkgmin,NaN,NaN,2166-11-08 15:00:00
4,3000000,30131,NaN,NaN,60.000000,mcgkgmin,NaN,NaN,2166-11-08 18:00:00
...,...,...,...,...,...,...,...,...,...
1545060,3038871,225158,151.326386,ml,34.005929,mL/hour,218.771470,33.975201,2131-09-24 09:12:00
1545061,3038871,221906,4.842444,mg,0.200182,mcg/kg/min,7.000688,0.200000,2131-09-24 09:12:00
1545062,3038871,220949,50.000000,ml,NaN,NaN,50.000000,0.000000,2131-09-24 09:52:00
1545063,3038871,225943,2.500000,ml,7.142857,mL/hour,2.500001,7.136620,2131-09-24 13:39:00


### Save

In [16]:
if not os.path.exists('./mimiciii'):
            os.makedirs('./mimiciii')

with open('./mimiciii/III_label.pickle', 'wb') as f:
    pickle.dump(III_label, f)
with open('./mimiciii/III_lab.pickle', 'wb') as f:
    pickle.dump(III_lab, f)
with open('./mimiciii/III_prescrip.pickle', 'wb') as f:
    pickle.dump(III_prescrip, f)
with open('./mimiciii/III_input.pickle', 'wb') as f:
    pickle.dump(III_input, f)
with open('./mimiciii/III_icustay.pickle', 'wb') as f:
    pickle.dump(III_icustay, f)


In [ ]:
if not os.path.exists('./mimiciv'):
            os.makedirs('./mimiciv')

with open('./mimiciv/IV_label.pickle', 'wb') as f:
    pickle.dump(IV_label, f)
with open('./mimiciv/IV_lab.pickle', 'wb') as f:
    pickle.dump(IV_lab, f)
with open('./mimiciv/IV_prescrip.pickle', 'wb') as f:
    pickle.dump(IV_prescrip, f)
with open('./mimiciv/IV_input.pickle', 'wb') as f:
    pickle.dump(IV_input, f)
with open('./mimiciv/IV_icustay.pickle', 'wb') as f:
    pickle.dump(IV_icustay, f)

In [174]:
with open('./mimiciii.pickle', 'rb') as f:
    data = pickle.load(f)

### Dictionary

In [137]:
III_label.ICUSTAY_ID.unique()

array([3013451, 3024613, 3003849, ..., 3030524, 3004964, 3025323])

In [116]:
def make_dict(df, charttime, columns, base_dict):
    
    for time in df[charttime].unique():
        base_dict[time] = []
        
        # if time not in list(base_dict.keys()):
        #     base_dict[time] = []
        
    for i in range(len(df)):
        for col in (columns):
            base_dict[df[charttime].iloc[i]].append(col)
            base_dict[df[charttime].iloc[i]].append(str(df[col].iloc[i]))
            
    base_dict = dict(sorted(base_dict.items()))
    
    return base_dict

def make_dict(df, charttime, columns, base_dict):
    
    for time in df[charttime].unique():
        base_dict[time] = {}
        
    for i in range(len(df)):
        for col in (columns):
            base_dict[df[charttime].iloc[i]][col] = df[col].iloc[i]
    
    return base_dict

In [117]:
III_lab_columns = ['ITEMID', 'VALUE', 'VALUEUOM', 'FLAG']
III_prescrip_columns = ['DRUG_TYPE', 'DRUG', 'PROD_STRENGTH', 'DOSE_VAL_RX', 'DOSE_UNIT_RX', 'FORM_VAL_DISP', 'FORM_UNIT_DISP', 'ROUTE']
III_input_columns = ['ITEMID', 'AMOUNT', 'AMOUNTUOM', 'ORIGINALAMOUNT']

for id in tqdm(III_label.ICUSTAY_ID.unique()):
    test = {}
    lab = III_lab[III_lab.ICUSTAY_ID == id]
    make_dict(lab, 'CHARTTIME', III_lab_columns, test)

In [118]:
III_dict_final = {}

for id in tqdm(III_label.ICUSTAY_ID.unique()):
    dict_by_icuid = {}
    lab = III_lab[III_lab.ICUSTAY_ID == id]
    prescrip = III_prescrip[III_prescrip.ICUSTAY_ID == id]
    input_ = III_input[III_input.ICUSTAY_ID == id]
    
    make_dict(lab, 'CHARTTIME', III_lab_columns, dict_by_icuid)
    make_dict(prescrip, 'STARTDATE', III_prescrip_columns, dict_by_icuid)
    make_dict(input_, 'CHARTTIME', III_input_columns, dict_by_icuid)
    III_dict_final[id] = dict_by_icuid


100%|██████████| 34987/34987 [06:47<00:00, 85.92it/s] 


In [119]:
len(III_dict_final.keys()), len(III_label)

(34987, 34987)

In [136]:
IV_lab_columns = ['itemid', 'value', 'valueuom', 'ref_range_lower', 'ref_range_upper', 'flag']
IV_prescrip_columns = ['pharmacy_id', 'drug_type', 'drug', 'formulary_drug_cd', 'gsn', 'ndc', 'prod_strength', 'dose_val_rx', 'dose_unit_rx', 'form_val_disp', 'form_unit_disp', 'route']
IV_input_columns = ['itemid', 'amount', 'amountuom', 'rate', 'rateuom', 'originalamount', 'originalrate']

In [137]:
IV_dict_final = {}

for id in tqdm(IV_label.stay_id.unique()):
    dict_by_icuid = {}
    lab = IV_lab[IV_lab.stay_id == id]
    prescrip = IV_prescrip[IV_prescrip.stay_id == id]
    input_ = IV_input[IV_input.stay_id == id]
    
    make_dict(lab, 'charttime', IV_lab_columns, dict_by_icuid)
    make_dict(prescrip, 'starttime', IV_prescrip_columns, dict_by_icuid)
    make_dict(input_, 'starttime', IV_input_columns, dict_by_icuid)
    IV_dict_final[id] = dict_by_icuid

100%|██████████| 46159/46159 [16:12<00:00, 47.48it/s]  


In [138]:
len(IV_dict_final.keys()), len(IV_label)

(46159, 46159)

In [143]:
with open('./mimiciii.pickle', 'wb') as f:
    pickle.dump(III_dict_final, f)
with open('./mimiciv.pickle', 'wb') as f:
    pickle.dump(IV_dict_final, f)